<a href="https://colab.research.google.com/github/ngusadeep/Qwen3-Swahili-Cognition/blob/main/finetuning-notebooks/Qwen3_14B_Swahili_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-tuning Qwen3 LLM for Enhanced Swahili Cognitive Capabilities**
## Pushing the frontier of Swahili language understanding across Text, Speech, and Vision

This notebook fine-tunes the Qwen3 LLM suite to enhance its Swahili cognitive capabilities using the **Swahili-Instructions** dataset from Kaggle. This dataset contains high-quality Swahili instruction-following examples that improve the model's conversational and instruction-following abilities in Swahili.

## Framework: Unsloth for Efficient Fine-tuning
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>



### Installation

In [ ]:
%%capture

import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install --quiet kagglehub

In [ ]:
# =============================================================================
# HUGGINGFACE TOKEN SETUP (Using Google Colab userdata)
# =============================================================================

try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_API_KEY')
    print("✅ Successfully loaded HuggingFace token from Colab userdata")
except:
    HF_TOKEN = None
    print("⚠️ Could not load HF_API_KEY from Colab userdata. Set it in Colab secrets if you need to use gated models or push to hub.")


### Load Qwen3 Model with Unsloth

We'll load the Qwen3-14B model using Unsloth's FastLanguageModel for efficient fine-tuning. The model will be loaded in 4-bit quantization to optimize memory usage in Google Colab.

In [ ]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # Qwen 14B 2x faster
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    token = HF_TOKEN if HF_TOKEN else None,  # Use token from Colab userdata if available
)

### Add LoRA Adapters for Efficient Fine-tuning

We now add LoRA (Low-Rank Adaptation) adapters so we only need to update 1 to 10% of all parameters! This makes fine-tuning much more memory-efficient while maintaining model performance.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

<a name="Test"></a>
### Test Model with Swahili Prompt (Before Fine-tuning)

Let's first test the base model's Swahili capabilities before fine-tuning to establish a baseline.

In [ ]:
# Test the base model with a Swahili prompt
messages = [
    {"role" : "user", "content" : "Eleza kwa ufupi kuhusu historia ya Kiswahili na umuhimu wake katika Afrika Mashariki."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False,
)

from transformers import TextStreamer
print("=== Base Model Response (Before Fine-tuning) ===")
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256,
    temperature = 0.7, top_p = 0.8, top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<a name="Data"></a>
### Data Preparation for Swahili Fine-tuning

We'll use the **Swahili-Instructions** dataset from Kaggle which contains high-quality Swahili instruction-following examples. This dataset is in instruction/input/output format and we'll convert it to ShareGPT-style conversations for training.

Let's download and prepare the Swahili dataset:

In [ ]:
# =============================================================================
# BLOCK 1: IMPORTS AND SETUP
# =============================================================================

import os
import json
import pandas as pd
from pathlib import Path
import kagglehub
from datasets import Dataset

print("✓ All required libraries imported successfully!")

### Download Swahili-Instructions Dataset from Kaggle


In [ ]:
# =============================================================================
# BLOCK 2: DIRECTORY STRUCTURE SETUP
# =============================================================================

# Create directories for organized storage
DATA_DIR = Path("./datasets")
KAGGLE_DIR = DATA_DIR / "kaggle"

# Create directories
for dir_path in [DATA_DIR, KAGGLE_DIR]:
    dir_path.mkdir(exist_ok=True, parents=True)

print("📁 Created directory structure:")
print(f"   Main data directory: {DATA_DIR}")
print(f"   Kaggle datasets: {KAGGLE_DIR}")
print("✅ Directory setup complete!")

### Download Swahili-Instructions Dataset

In [ ]:
# =============================================================================
# BLOCK 3: KAGGLE DOWNLOAD HELPER FUNCTION
# =============================================================================

def download_kaggle_with_hub(dataset_id, local_name):
    """Download Kaggle dataset using kagglehub"""
    try:
        print(f"📥 Downloading {dataset_id}...")
        dataset_path = kagglehub.dataset_download(dataset_id)
        print(f"✅ Downloaded to: {dataset_path}")

        # Create symlink or copy to our organized structure
        target_path = KAGGLE_DIR / local_name
        if target_path.exists():
            import shutil
            shutil.rmtree(target_path)

        # Create symlink for efficient access
        try:
            target_path.symlink_to(Path(dataset_path), target_is_directory=True)
            print(f"🔗 Linked to: {target_path}")
        except OSError:
            # If symlink fails, copy the data
            import shutil
            shutil.copytree(dataset_path, target_path)
            print(f"📁 Copied to: {target_path}")

        return str(dataset_path), target_path
    except Exception as e:
        print(f"❌ Error downloading {dataset_id}: {e}")
        return None, None

print("✅ Kaggle download helper function ready!")

### Download Swahili-Instructions Dataset

In [ ]:
print("🚀 Starting Kaggle dataset download...")

# Download Swahili Instructions Dataset
print("\n" + "="*50)
print("📚 SWAHILI INSTRUCTIONS DATASET")
print("="*50)
swahili_instructions_path, swahili_instructions_dir = download_kaggle_with_hub(
    "alfaxadeyembe/swahili-instructions", "swahili_instructions"
)

print("\n✅ Kaggle dataset download completed!")

### Load and Examine Swahili-Instructions Dataset

In [ ]:
# =============================================================================
# BLOCK 4: LOAD SWAHILI INSTRUCTIONS DATA
# =============================================================================

print("📖 Reading Swahili Instructions Dataset...")

# Load the JSON file
swahili_instructions_path = Path("datasets/kaggle/swahili_instructions")
json_files = list(swahili_instructions_path.glob("*.json"))

if json_files:
    json_file = json_files[0]
    print(f"✅ Found JSON file: {json_file}")

    # Read the JSON data
    with open(json_file, 'r', encoding='utf-8') as f:
        swahili_data = json.load(f)

    print(f"✅ Successfully loaded JSON data")
    print(f"📊 Data type: {type(swahili_data)}")

    # If it's a list, show first few items
    if isinstance(swahili_data, list):
        print(f"📈 Number of entries: {len(swahili_data)}")
        print(f"\n📋 First entry structure:")
        if swahili_data:
            first_item = swahili_data[0]
            print(f"   Keys: {list(first_item.keys()) if isinstance(first_item, dict) else 'Not a dict'}")
            print(f"   Sample: {first_item}")

            # Show a few more samples to understand the structure
            print(f"\n📝 Sample entries (first 3):")
            for i, item in enumerate(swahili_data[:3]):
                print(f"   Entry {i+1}: {item}")

    # If it's a dict, show the structure
    elif isinstance(swahili_data, dict):
        print(f"📚 Dictionary keys: {list(swahili_data.keys())}")
        for key, value in swahili_data.items():
            print(f"   {key}: {type(value)} - {len(value) if hasattr(value, '__len__') else value}")

else:
    print("❌ No JSON files found in the directory")
    swahili_data = None

### Convert to ShareGPT Format and Standardize

In [ ]:
# =============================================================================
# BLOCK 5: STANDARDIZE DATA FORMATS USING UNSLOTH
# =============================================================================

from unsloth.chat_templates import standardize_data_formats

print("🔄 Converting Swahili instructions to standardized format...")

# Convert the raw JSON data to ShareGPT format first
def convert_to_conversations_format(data):
    """Convert instruction-input-output format to conversations format"""
    conversations_data = []

    for item in data:
        # Create the user message
        user_message = item['instruction']

        # If there's input content, add it to the instruction
        if item.get('input', '').strip():
            user_message += f"\n\n{item['input']}"

        # Create conversation structure
        conversation = {
            "conversations": [
                {"from": "human", "value": user_message},
                {"from": "gpt", "value": item['output']}
            ]
        }

        # Keep the ID for reference
        if 'id' in item:
            conversation['id'] = item['id']

        conversations_data.append(conversation)

    return conversations_data

# Convert to conversations format
print(f"📝 Converting {len(swahili_data)} entries to conversations format...")
conversations_data = convert_to_conversations_format(swahili_data)

# Create HuggingFace dataset
dataset = Dataset.from_list(conversations_data)
print(f"✅ Created dataset with {len(dataset)} conversations")

# Apply unsloth's standardize_data_formats
print("🔧 Applying unsloth's standardize_data_formats...")
try:
    dataset = standardize_data_formats(dataset)
    print("✅ Data standardization completed!")

    # Print sample to see the structure after standardization
    print(f"\n📊 Sample after standardization:")
    print(f"Dataset features: {dataset.features}")

    if len(dataset) > 0:
        sample = dataset[0]
        print(f"\n📋 Sample entry structure:")
        for key, value in sample.items():
            if isinstance(value, list) and len(value) > 0:
                print(f"  {key}: {value}")
            elif isinstance(value, str) and len(value) > 200:
                print(f"  {key}: {value[:200]}...")
            else:
                print(f"  {key}: {value}")

        print(f"\n💬 Sample conversation:")
        if 'conversations' in sample:
            for i, turn in enumerate(sample['conversations']):
                # Handle the new format: role/content instead of from/value
                role = turn.get('role', turn.get('from', 'unknown'))
                content = turn.get('content', turn.get('value', 'no content'))
                print(f"  Turn {i+1} ({role}): {content[:150]}...")

    print(f"\n✅ Dataset ready for chat template application!")
    print(f"   Total conversations: {len(dataset)}")
    print(f"   Features: {list(dataset.features.keys())}")

except Exception as e:
    print(f"❌ Error during standardization: {e}")
    print("Dataset created but standardization failed. You may proceed with manual formatting.")
    raise


In [ ]:
### Apply Qwen3 Chat Template

Now we'll apply the Qwen3 chat template to format the conversations for training:

In [ ]:
# Apply Qwen3 chat template to the dataset
print("🎨 Applying Qwen3 chat template...")
conversations_text = tokenizer.apply_chat_template(
    list(dataset["conversations"]),
    tokenize = False,
)

print(f"✅ Applied chat template to {len(conversations_text)} conversations")
print(f"\n📝 Sample formatted conversation (first 500 chars):")
print(conversations_text[0][:500] + "..." if len(conversations_text[0]) > 500 else conversations_text[0])

### Create Final Training Dataset

In [ ]:
# Convert to pandas Series and create final dataset
import pandas as pd

data = pd.Series(conversations_text)
data.name = "text"

# Create HuggingFace dataset
combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed = 3407)

print(f"✅ Final training dataset created!")
print(f"   Total examples: {len(combined_dataset)}")
print(f"   Dataset features: {combined_dataset.features}")

### Preview Final Dataset

In [ ]:
# Preview a sample from the final dataset
print("📖 Sample from final training dataset:")
print("="*70)
sample_text = combined_dataset[0]["text"]
print(sample_text[:800] + "..." if len(sample_text) > 800 else sample_text)
print("="*70)

The dataset is now ready for training! We have successfully:
1. ✅ Downloaded the Swahili-Instructions dataset from Kaggle
2. ✅ Converted instruction/input/output format to ShareGPT conversations format
3. ✅ Standardized the data using Unsloth's `standardize_data_formats`
4. ✅ Applied Qwen3 chat template
5. ✅ Created final training dataset with shuffled examples

In [ ]:
# Dataset is ready! The combined_dataset variable is already created above.
# You can now proceed to training.

<a name="Train"></a>
### Fine-tune Qwen3 for Swahili

Now let's train our model on the Swahili datasets. We'll do 30 steps initially for testing, but you can set `num_train_epochs=1` for a full training run, and set `max_steps=None` to disable step limiting.

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

### Start Swahili Fine-tuning

Let's begin training the model on Swahili data! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Test Fine-tuned Model with Swahili Prompts

Let's test our fine-tuned model with Swahili prompts! According to the Qwen-3 team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`. For normal chat-based inference, use `temperature = 0.7, top_p = 0.8, top_k = 20`.

In [ ]:
# Test with Swahili conversational prompt (non-thinking mode)
messages = [
    {"role" : "user", "content" : "Eleza kwa ufupi kuhusu umuhimu wa lugha ya Kiswahili katika Afrika Mashariki na jinsi inavyotumika katika maisha ya kila siku."}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
print("=== Fine-tuned Model Response (Swahili - Non-Thinking Mode) ===")
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

In [ ]:
# Test with Swahili reasoning prompt (thinking mode enabled)
messages = [
    {"role" : "user", "content" : "Kama una shilingi 5000 na unataka kununua vitabu viwili: moja ni shilingi 2000 na nyingine ni shilingi 3500. Je, utabaki na pesa ngapi?"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Enable thinking for reasoning
)

from transformers import TextStreamer
print("\n=== Fine-tuned Model Response (Swahili - Thinking Mode) ===")
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    temperature = 0.6, top_p = 0.95, top_k = 20, # For thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<a name="Save"></a>
### Save Fine-tuned Swahili Model

To save the final fine-tuned Swahili model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = HF_TOKEN) # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = HF_TOKEN) # Online saving


### Load Saved Swahili Model

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

### Save Swahili Model in Different Formats

We also support saving to `float16` directly for VLLM deployment. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! The token is automatically loaded from Colab userdata (HF_API_KEY).

In [ ]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = HF_TOKEN)

# Merge to 4bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = HF_TOKEN)

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False: # Pushing to HF Hub
    model.push_to_hub("hf/model", token = HF_TOKEN)
    tokenizer.push_to_hub("hf/model", token = HF_TOKEN)


### Export Swahili Model to GGUF / llama.cpp Format

To save your fine-tuned Swahili model to `GGUF` / `llama.cpp` format, we support it natively! We clone `llama.cpp` and default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# Token is loaded from Colab userdata (HF_API_KEY)
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = HF_TOKEN)

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = HF_TOKEN)

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = HF_TOKEN)

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = HF_TOKEN, # Token loaded from Colab userdata (HF_API_KEY)
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

## Summary

We've successfully fine-tuned the Qwen3-14B model for enhanced Swahili cognitive capabilities using the Nadhari/Swahili-Thinking datasets. The model now has improved:
- **Text understanding** in Swahili
- **Reasoning capabilities** for Swahili problem-solving
- **Conversational abilities** in Swahili

The fine-tuned model can be deployed for various Swahili language applications across African countries and globally.

---

## Resources and Support

If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other useful links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
4. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).
